In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 1 : formalization: get started with Vertex AI Data Labeling

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_with_data_labeling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_with_data_labeling.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 1 : data management: get started with Vertex AI Data Labeling service.

### Dataset

The dataset used for this tutorial is the [Flowers dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket. The trained model predicts the type of flower an image is from a class of five flowers: daisy, dandelion, rose, sunflower, or tulip.

### Objective

In this tutorial, you learn how to use the `Vertex AI Data Labeling` service/

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Data Labeling`
- `Vertex AI Dataset`

The steps performed include:

- Create a Specialist Pool for data labelers.
- Create a data labeling job.
- Submit the data labeling job.
- List data labeling jobs.
- Cancel a data labeling job.

Learn more about [Request a Vertex AI Data Labeling job](https://cloud.google.com/vertex-ai/docs/datasets/data-labeling-job).

### Costs 


This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage


Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages to execute this notebook.

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

# Install the packages
! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG
! pip3 install --upgrade google-cloud-storage $USER_FLAG

### Restart the Kernel

Once you've installed the Vertex AI SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.


In [2]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex AI APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.


In [25]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [26]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

E0330 08:06:57.201329997   15225 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Project ID: vertex-ai-dev


In [27]:
! gcloud config set project $PROJECT_ID

E0330 08:07:00.217158024   15225 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [28]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.


In [29]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

#### Email

You need an email address to send labeling job request to. This is the email address will be the manager of the data labeling specialist pool.

In this tutorial, if you don't specify an email address, the email address associated with your project ID will be used.

In [30]:
EMAIL = "[your-email-address]"  # @param {type: "string"}

In [31]:
if EMAIL == "[your-email-address]":
    shell_output = ! gcloud auth list 2>/dev/null
    EMAIL = shell_output[2].replace("*", "").strip()

print(EMAIL)

E0330 08:07:12.767462063   15225 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


931647533046-compute@developer.gserviceaccount.com


### Authenticate your GCP account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an Vertex AI Workbench notebook and run the cell, the cell knows to skip executing the authentication steps.*


In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex AI, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.


In [32]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [33]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.


In [34]:
! gsutil mb -l $REGION $BUCKET_URI

E0330 08:07:20.445481351   15225 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Creating gs://vertex-ai-devaip-20220330080709/...


Finally, validate access to your Cloud Storage bucket by examining its contents:


In [35]:
! gsutil ls -al $BUCKET_URI

E0330 08:07:24.138360770   15225 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants


#### Import Vertex AI SDK

Import the Vertex AI SDK into our Python environment.


In [36]:
import os
import sys
import time

import google.cloud.aiplatform as aip
from google.cloud import storage
from google.cloud.aiplatform import gapic
from google.protobuf.json_format import ParseDict
from google.protobuf.struct_pb2 import Value


#### Vertex AI constants

Setup up the following constants for Vertex AI:

- `API_ENDPOINT`: The Vertex AI API service endpoint for dataset, model, job, pipeline and endpoint services.
- `PARENT`: The Vertex AI location root path for dataset, model and endpoint resources.

In [37]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### Schema constants

Next, setup constants for schemas related image classification datasets:

- Data Labeling (Annotations) Schemas: Tells the managed dataset service how the data is labeled (annotated).

In [38]:
# Image labeling task
LABELING_SCHEMA_IMAGE = "gs://google-cloud-aiplatform/schema/datalabelingjob/inputs/image_classification_1.0.0.yaml"

## Create clients

The Vertex AI SDK works as a client/server model. On your side (the Python script) you create a client that sends requests and receives responses from the server (Vertex AI).

You use several clients in this tutorial, so set them all up upfront.

- Specialist pool service for specialist pools
- Job Service for data labeling


In [39]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}

clients = {}
clients["job"] = gapic.JobServiceClient(client_options=client_options)

# add client for specialist pool
clients["specialist_pool"] = gapic.SpecialistPoolServiceClient(
    client_options=client_options
)

for client in clients.items():
    print(client)

('job', <google.cloud.aiplatform_v1.services.job_service.client.JobServiceClient object at 0x7f6d23c6a6d0>)
('specialist_pool', <google.cloud.aiplatform_v1.services.specialist_pool_service.client.SpecialistPoolServiceClient object at 0x7f6d5e470400>)


### Create a CSV file for examples to label

Next, you will create a CSV file for the examples you are requesting to be labeled. 

In this example, the examples to label are images. For each row in the CSV file, you specify the Cloud Storage location of the image to label.

In [40]:
test_filename='labeling.csv'
LABELING_FILES = [
    "gs://cloud-samples-data/vision/automl_classification/flowers/daisy/100080576_f52e8ee070_n.jpg",
    "gs://cloud-samples-data/vision/automl_classification/flowers/daisy/102841525_bd6628ae3c.jpg",
]

IMPORT_FILE = BUCKET_URI + "/labeling.csv"

bucket = storage.Client(project=PROJECT_ID).bucket(BUCKET_URI.replace("gs://", ""))

# creating a blob
blob = bucket.blob(blob_name=test_filename)

# creating data variable
data = LABELING_FILES[0] + "\n" + LABELING_FILES[1] + "\n"

# uploading data variable content to bucket
blob.upload_from_string(data,content_type='text/csv')

# printing path of uploaded file
print(IMPORT_FILE)

# printing content of uploaded file
! gsutil cat $IMPORT_FILE

gs://vertex-ai-devaip-20220330080709/labeling.csv


E0330 08:07:37.984447577   15225 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


gs://cloud-samples-data/vision/automl_classification/flowers/daisy/100080576_f52e8ee070_n.jpg
gs://cloud-samples-data/vision/automl_classification/flowers/daisy/102841525_bd6628ae3c.jpg


## Create a unlabeled dataset

Next, you create a dataset for the data to be labeled.

In [41]:
dataset = aip.ImageDataset.create("labeling_" + TIMESTAMP)
print(dataset)

INFO:google.cloud.aiplatform.datasets.dataset:Creating ImageDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create ImageDataset backing LRO: projects/931647533046/locations/us-central1/datasets/3776914800387293184/operations/3121085320196521984
INFO:google.cloud.aiplatform.datasets.dataset:ImageDataset created. Resource name: projects/931647533046/locations/us-central1/datasets/3776914800387293184
INFO:google.cloud.aiplatform.datasets.dataset:To use this ImageDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.ImageDataset('projects/931647533046/locations/us-central1/datasets/3776914800387293184')
resource name: projects/931647533046/locations/us-central1/datasets/3776914800387293184


## Import the unlabeled data

Now, import the unlabeled data to the dataset, i.e., the examples to be labeled.

In [42]:
dataset.import_data(
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.image.single_label_classification,
)

INFO:google.cloud.aiplatform.datasets.dataset:Importing ImageDataset data: projects/931647533046/locations/us-central1/datasets/3776914800387293184
INFO:google.cloud.aiplatform.datasets.dataset:Import ImageDataset data backing LRO: projects/931647533046/locations/us-central1/datasets/3776914800387293184/operations/5848014894569357312
INFO:google.cloud.aiplatform.datasets.dataset:ImageDataset data imported. Resource name: projects/931647533046/locations/us-central1/datasets/3776914800387293184


resource name: projects/931647533046/locations/us-central1/datasets/3776914800387293184

## Create a new data specialist pool

Your data labeling job will be sent to a data specialist pool. You may have one or more multiple specialist pools. 

In this next step, you create a new specialist pool with the method `create_specialist_pool()`. The request includes the parameters:

- `name`: The resource name of the specialist pool.
- `display_name`: A human readable name for the specialist pool.
- `specialist_manager_emails`: A list of the email addresses of the manager(s) for the specialist pool.

*Note:* You can use an existing specialist pool if one already existed.

In [43]:
specialist_pool = {
    "name": "labeling_" + TIMESTAMP,
    "display_name": "labeling_" + TIMESTAMP,
    "specialist_manager_emails": [EMAIL],
}

request = clients["specialist_pool"].create_specialist_pool(
    parent=PARENT, specialist_pool=specialist_pool
)

result = request.result()
print(result)

specialist_name = result.name

specialist_id = specialist_name.split("/")[-1]

print(specialist_name)

name: "projects/931647533046/locations/us-central1/specialistPools/246114682761379840"

projects/931647533046/locations/us-central1/specialistPools/246114682761379840


## Create data labeling job

Now that you have a specialist pool, you can send a data labeling request using the `create_data_labeling_job()` method.

Your request will consist of the following:

- The Vertex AI Dataset with the unlabeled data.
- Instructions for labeling.

In [44]:
# create placeholder file for instructions for data labeling
! echo "this is instruction" >> instruction.txt | gsutil cp instruction.txt $BUCKET_URI

E0330 08:09:35.711555974   15225 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Copying file://instruction.txt [Content-Type=text/plain]...
/ [1 files][   40.0 B/   40.0 B]                                                
Operation completed over 1 objects/40.0 B.                                       


In [45]:
LABLEING_SCHEMA = LABELING_SCHEMA_IMAGE
INSTRUCTION_FILE = BUCKET_URI + "/instruction.txt"

inputs = ParseDict({"annotation_specs": ["rose"]}, Value())

data_labeling_job = {
    "display_name": "labeling_" + TIMESTAMP,
    "datasets": [dataset.resource_name],
    "labeler_count": 1,
    "instruction_uri": INSTRUCTION_FILE,
    "inputs_schema_uri": LABLEING_SCHEMA,
    "inputs": inputs,
    "annotation_labels": {
        "aiplatform.googleapis.com/annotation_set_name": "data_labeling_job_specialist_pool"
    },
    "specialist_pools": [specialist_name],
}

print(data_labeling_job)

request = clients["job"].create_data_labeling_job(
    parent=PARENT, data_labeling_job=data_labeling_job
)

print(request)

labeling_task_name = request.name

print(labeling_task_name)

{'display_name': 'labeling_20220330080709', 'datasets': ['projects/931647533046/locations/us-central1/datasets/3776914800387293184'], 'labeler_count': 1, 'instruction_uri': 'gs://vertex-ai-devaip-20220330080709/instruction.txt', 'inputs_schema_uri': 'gs://google-cloud-aiplatform/schema/datalabelingjob/inputs/image_classification_1.0.0.yaml', 'inputs': struct_value {
  fields {
    key: "annotation_specs"
    value {
      list_value {
        values {
          string_value: "rose"
        }
      }
    }
  }
}
, 'annotation_labels': {'aiplatform.googleapis.com/annotation_set_name': 'data_labeling_job_specialist_pool'}, 'specialist_pools': ['projects/931647533046/locations/us-central1/specialistPools/246114682761379840']}
name: "projects/931647533046/locations/us-central1/dataLabelingJobs/4032309911042916352"
display_name: "labeling_20220330080709"
datasets: "projects/931647533046/locations/us-central1/datasets/3776914800387293184"
labeler_count: 1
instruction_uri: "gs://vertex-ai-deva

### Get a data labeling job

You can get information on your data labeling job using the `get_data_labeling_job()` method, with the following parameters:

- `name`: The name of the labeling task.

In [46]:
request = clients["job"].get_data_labeling_job(name=labeling_task_name)
print(request)

name: "projects/931647533046/locations/us-central1/dataLabelingJobs/4032309911042916352"
display_name: "labeling_20220330080709"
datasets: "projects/931647533046/locations/us-central1/datasets/3776914800387293184"
labeler_count: 1
instruction_uri: "gs://vertex-ai-devaip-20220330080709/instruction.txt"
inputs_schema_uri: "gs://google-cloud-aiplatform/schema/datalabelingjob/inputs/image_classification_1.0.0.yaml"
inputs {
  struct_value {
    fields {
      key: "annotationSpecs"
      value {
        list_value {
          values {
            string_value: "rose"
          }
        }
      }
    }
  }
}
state: JOB_STATE_RUNNING
create_time {
  seconds: 1648627781
  nanos: 865381000
}
update_time {
  seconds: 1648627782
  nanos: 393063000
}
specialist_pools: "projects/931647533046/locations/us-central1/specialistPools/246114682761379840"



### Cancel a data labeling task

You can cancel a data labeling request using the `cancel_data_labeling_job()` method, with the following parameters:

- `name`: The name of the labeling task.

In [47]:
request = clients["job"].cancel_data_labeling_job(name=labeling_task_name)
print(request)

None


### Wait for labeling job to be canceled

The cancel request is asyncrhonous. The code below polls on the labeling job status until the status is CANCELED.

In [48]:
while True:
    response = clients["job"].get_data_labeling_job(name=labeling_task_name)
    if response.state == gapic.JobState.JOB_STATE_CANCELLED:
        print("Labeling job CANCELED")
        break
    else:
        print("Canceling labeling job:", response.state)
        time.sleep(60)

Canceling labeling job: JobState.JOB_STATE_RUNNING
Labeling job CANCELED


# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [49]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

# Delete the dataset using the Vertex AI fully qualified identifier for the dataset
dataset.delete()

# Delete the labeling job using the Vertex AI fully qualified identifier for the dataset
request = clients["job"].delete_data_labeling_job(name=labeling_task_name)

# Delete the specialist pool using the Vertex AI fully qualified identifier for the dataset
clients["specialist_pool"].delete_specialist_pool(name=specialist_name)

#Delete the bucket created
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

INFO:google.cloud.aiplatform.base:Deleting ImageDataset : projects/931647533046/locations/us-central1/datasets/3776914800387293184
INFO:google.cloud.aiplatform.base:Delete ImageDataset  backing LRO: projects/931647533046/locations/us-central1/operations/469591039582142464
INFO:google.cloud.aiplatform.base:ImageDataset deleted. . Resource name: projects/931647533046/locations/us-central1/datasets/3776914800387293184


E0330 08:12:14.893855329   15225 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Removing gs://vertex-ai-devaip-20220330080709/instruction.txt#1648627777064720...
Removing gs://vertex-ai-devaip-20220330080709/labeling.csv#1648627657936184...  
/ [2 objects]                                                                   
Operation completed over 2 objects.                                              
Removing gs://vertex-ai-devaip-20220330080709/...
